In [ ]:
!pip install pymorphy2
! pip install -U adapter-transformers
!pip install datasets

In [ ]:
import pandas as pd
from datasets import load_dataset
from drive.MyDrive.xmas_hack import preprocess
import torch
import json
from transformers import RobertaConfig, RobertaModelWithHeads, EvalPrediction, TrainingArguments, AdapterTrainer, RobertaTokenizer
from tqdm.notebook import tqdm
import sklearn.metrics as metrics

In [ ]:
!unzip /content/drive/MyDrive/xmas_hack/hacka-aka-embedika.zip

In [ ]:
with open("/content/classes.json", "r") as classes:
    classes = json.load(classes)

In [ ]:
data = {'id': classes.keys(),
        'label': classes.values()}
df = pd.DataFrame.from_dict(data)
df = df.sort_values(by='id')
df = df.reset_index()
del df['index']
df

,id,label
0,02682d726b725f95b9ee85f751c043d0.doc,Договоры для акселератора/Договоры оказания услуг
1,03a70fe60be9ecc2a63798a361fc9689.docx,Договоры для акселератора/Договоры подряда
2,073a0d372820c3c2bffe9ba24a7ed7af.doc,Договоры для акселератора/Договоры купли-продажи
3,084edc7dfc3db04e3a3c55c102f47bcb.doc,Договоры для акселератора/Договоры аренды
4,086ad48895d2a73854b6151decc28800.pdf,Договоры для акселератора/Договоры купли-продажи
...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.pdf,Договоры для акселератора/Договоры купли-продажи
116,f6377999f8a5aa9a09b03e428ac93153.doc,Договоры для акселератора/Договоры поставки
117,f8eabec67b770c384d84f9feb7e02543.doc,Договоры для акселератора/Договоры оказания услуг
118,f96c2a58215aa54aeb6713cbb457fad9.doc,Договоры для акселератора/Договоры оказания услуг


In [ ]:
for i in range(len(df['id'])):
  df['id'][i] = df['id'][i].split('.')[0] + '.txt'

In [ ]:
texts_id = list(df['id'])
df['text'] = 0
df['text_norm'] = 0

In [ ]:
for i in range(0, len(df)):
  with open("/content/drive/MyDrive/xmas_hack/txt_data/" + texts_id[i], "r", encoding='utf-8') as text:
    text = text.read()
    df['text'][i] = text
df

<ipython-input-21-b83594f0df20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = text
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,label,test,text_norm,text
0,02682d726b725f95b9ee85f751c043d0.txt,Договоры для акселератора/Договоры оказания услуг,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
1,03a70fe60be9ecc2a63798a361fc9689.txt,Договоры для акселератора/Договоры подряда,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
2,073a0d372820c3c2bffe9ba24a7ed7af.txt,Договоры для акселератора/Договоры купли-продажи,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
3,084edc7dfc3db04e3a3c55c102f47bcb.txt,Договоры для акселератора/Договоры аренды,0,0,﻿5\n\nEvaluation Only. Created with Aspose.Wor...
4,086ad48895d2a73854b6151decc28800.txt,Договоры для акселератора/Договоры купли-продажи,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
...,...,...,...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.txt,Договоры для акселератора/Договоры купли-продажи,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
116,f6377999f8a5aa9a09b03e428ac93153.txt,Договоры для акселератора/Договоры поставки,0,0,﻿\nEvaluation Only. Created with Aspose.Words....
117,f8eabec67b770c384d84f9feb7e02543.txt,Договоры для акселератора/Договоры оказания услуг,0,0,﻿Форма 5.3.\nEvaluation Only. Created with Asp...
118,f96c2a58215aa54aeb6713cbb457fad9.txt,Договоры для акселератора/Договоры оказания услуг,0,0,﻿ФОРМА 5.4. ...


In [ ]:
for i in range(len(df)):
  df['text'][i] = df['text'][i].replace("Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.", "")
  test_text = preprocess.preprocess_pipeline(df['text'][i])
  df['text_norm'][i] = test_text

In [ ]:
df

,id,label,test,text_norm,text
0,02682d726b725f95b9ee85f751c043d0.txt,Договоры для акселератора/Договоры оказания услуг,0,﻿ договор г . москва « _ » _ 2020 года гр . рф...,﻿\n\nДОГОВОР\n\nг. Москва\n«_____» ___________...
1,03a70fe60be9ecc2a63798a361fc9689.txt,Договоры для акселератора/Договоры подряда,0,"﻿ договор № _ от « _ » _ г . г . _ , именуемое...",﻿\n\nДОГОВОР \n\n№ ______ от «___» ___________...
2,073a0d372820c3c2bffe9ba24a7ed7af.txt,Договоры для акселератора/Договоры купли-продажи,0,"﻿ договор московская область , город _ , девят...","﻿\n\n\n\nДОГОВОР \n\nМосковская область, город..."
3,084edc7dfc3db04e3a3c55c102f47bcb.txt,Договоры для акселератора/Договоры аренды,0,﻿ 5 договор № _ г . _ 200_ года на основании п...,﻿5\n\n\n\nДОГОВОР № __________\n\nг. _________...
4,086ad48895d2a73854b6151decc28800.txt,Договоры для акселератора/Договоры купли-продажи,0,﻿ ( типовая форма ) договор купли - продажи № ...,﻿\n\n(Типовая форма) ДОГО...
...,...,...,...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.txt,Договоры для акселератора/Договоры купли-продажи,0,"﻿ договор купли - продажи ………. оборудования , ...",﻿\n\nДОГОВОР КУПЛИ-ПРОДАЖИ ………. О...
116,f6377999f8a5aa9a09b03e428ac93153.txt,Договоры для акселератора/Договоры поставки,0,"﻿ договор г . _ "" _ "" _20_г . предприятие _ , ...",﻿\n\nДОГОВОР \n\n\n г._____________________...
117,f8eabec67b770c384d84f9feb7e02543.txt,Договоры для акселератора/Договоры оказания услуг,0,﻿ форма 5 . 3 . приложение № 10 договор № _ г ...,﻿Форма 5.3.\n\nПриложение № 10\n\nДОГОВОР № __...
118,f96c2a58215aa54aeb6713cbb457fad9.txt,Договоры для акселератора/Договоры оказания услуг,0,﻿ форма 5 . 4 . приложение № 11 к приказу № мм...,﻿ФОРМА 5.4. ...


In [ ]:
labels = {'0': 'Договоры для акселератора/Договоры поставки',
 '1': 'Договоры для акселератора/Договоры оказания услуг',
 '2': 'Договоры для акселератора/Договоры подряда',
 '3': 'Договоры для акселератора/Договоры аренды',
 '4': 'Договоры для акселератора/Договоры купли-продажи'}

In [ ]:
labels = {v:k for k, v in labels.items()}

In [ ]:
for i in range(len(df)):
  print(df['label'][i])
  df['label'][i] = labels[df['label'][i]]

In [ ]:
del df['text']

In [ ]:
df

,id,label,text_norm
0,02682d726b725f95b9ee85f751c043d0.txt,1,﻿ договор г . москва « _ » _ 2020 года гр . рф...
1,03a70fe60be9ecc2a63798a361fc9689.txt,2,"﻿ договор № _ от « _ » _ г . г . _ , именуемое..."
2,073a0d372820c3c2bffe9ba24a7ed7af.txt,4,"﻿ договор московская область , город _ , девят..."
3,084edc7dfc3db04e3a3c55c102f47bcb.txt,3,﻿ 5 договор № _ г . _ 200_ года на основании п...
4,086ad48895d2a73854b6151decc28800.txt,4,﻿ ( типовая форма ) договор купли - продажи № ...
...,...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.txt,4,"﻿ договор купли - продажи ………. оборудования , ..."
116,f6377999f8a5aa9a09b03e428ac93153.txt,0,"﻿ договор г . _ "" _ "" _20_г . предприятие _ , ..."
117,f8eabec67b770c384d84f9feb7e02543.txt,1,﻿ форма 5 . 3 . приложение № 10 договор № _ г ...
118,f96c2a58215aa54aeb6713cbb457fad9.txt,1,﻿ форма 5 . 4 . приложение № 11 к приказу № мм...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text_norm'], df['label'], test_size=0.25, random_state=42)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')

**Train dataset**

In [ ]:
data =  {'text_norm': X_train,
        'label': y_train}
dataset = pd.DataFrame.from_dict(data)
dataset

In [ ]:
dataset.to_csv('train_docs_80.csv')

In [ ]:
dataset = load_dataset('csv', data_files={'train': '/content/train_docs_80.csv'})

**Test dataset**

In [ ]:
test_data =  {'text_norm': X_test,
        'label': y_test}
test_dataset = pd.DataFrame.from_dict(test_data)
test_dataset

,text_norm,label
44,"﻿ договор г . _ « _ » _200_г . гражданин _ , п...",3
47,"﻿ договор г . _ « _ » _200_ года . _ , именуем...",3
4,﻿ ( типовая форма ) договор купли - продажи № ...,4
55,﻿ договор 21 . 11 . 2011 № 5 г . москва общест...,2
26,"﻿ договор поставки № _ рф , г . челябинск "" _ ...",0
64,﻿ договор закупки № _ / г . « _ » _ 2019 г . о...,0
73,﻿ договор № цкп . _ г . москва « _ » _ 2018 г ...,1
10,"﻿ договор № _ дата , место заключения ( населе...",0
40,﻿ 1 проект договора договор № _ г . ставрополь...,0
107,﻿ форма 2 . 2 приложение № 4 к приказу № _от _...,3


**Train set preprocess**

In [ ]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  print(batch.keys())
  return tokenizer(batch["text_norm"], max_length=80, truncation=True, padding="max_length")

In [ ]:
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
dataset = dataset.rename_column("text_norm", "text")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  0%|          | 0/1 [00:00<?, ?ba/s]

KeysView({'Unnamed: 0': [42, 12, 15, 114, 76, 97, 24, 78, 22, 96, 56, 110, 30, 53, 118, 9, 33, 25, 69, 28, 98, 85, 5, 90, 68, 39, 49, 35, 16, 66, 34, 113, 7, 43, 72, 67, 83, 27, 19, 95, 100, 8, 13, 84, 3, 17, 38, 117, 6, 77, 111, 94, 54, 50, 80, 46, 81, 61, 116, 79, 93, 41, 58, 48, 101, 57, 75, 32, 112, 59, 63, 105, 37, 29, 115, 1, 52, 21, 2, 23, 103, 99, 87, 108, 74, 86, 82, 119, 20, 60, 71, 106, 14, 92, 51, 102], 'text_norm': ['\ufeff договор г . _ « _ » _ 200_г . _ », именуемое в дальнейшем продавец в лице генерального директора _ и _ , именуемое в дальнейшем покупатель в лице _ , действующего на основании _ , с другой стороны , заключили настоящий договор о нижеследующем : 1 . предмет договора 1 . 1 . продавец обязуется произвести и передать , а покупатель принять и оплатить товары в количестве и ассортименте , указанном продавцом в заказе . 1 . 2 . заказ , согласовывается сторонами , и оформляется в виде приложений к настоящему договору . согласование заказа допускается посредство

**Test set preprocess**

**Fit**

In [ ]:
MODEL_NAME = 'cointegrated/rubert-tiny2'

In [ ]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    MODEL_NAME,
    config=config,
)

loading configuration file https://huggingface.co/sberbank-ai/ruRoberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/191cf2edd2e8d2007f9a39479b45315ac1b79fe23fa4786511de1c8f9b9def46.ebd71ffa9cb74c7d1d39cf67343e2ae8465e2091f9fb70dbd5dec0cf58b1b60b
Model config RobertaConfig {
  "_name_or_path": "/home/jovyan/models/roberta/roberta_l",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "voca

In [ ]:
# Add a new adapter
model.add_adapter("docs")
# Add a matching classification head
model.add_classification_head(
    "docs",
    num_labels=5,
    id2label={0: 'Deliveries',
      1: 'Service_provision',
      2: 'Agreements',
      3: 'Rental',
      4: 'Purchase_and_sale'
 }
  )
# Activate the adapter
model.train_adapter("docs")

Adding adapter 'docs'.
Adding head 'docs' with config {'head_type': 'classification', 'num_labels': 5, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'Deliveries': 0, 'Service_provision': 1, 'Agreements': 2, 'Rental': 3, 'Purchase_and_sale': 4}, 'use_pooler': False, 'bias': True}.


In [ ]:
from transformers import TrainingArguments, AdapterTrainer
import torch

torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=18,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    output_dir="./training_output",
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    
)

def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
     #compute_metrics=F1Score.compute_f1,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: Unnamed: 0, text. If Unnamed: 0, text are not expected by `RobertaModelWithHeads.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 96
  Num Epochs = 18
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 108


Step,Training Loss
10,1.645700
20,1.574600
30,1.414400
40,1.278900
50,1.063900
60,0.811100
70,0.629000
80,0.521800
90,0.411100
100,0.380400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=108, training_loss=0.9282669204252737, metrics={'train_runtime': 47.0444, 'train_samples_per_second': 36.731, 'train_steps_per_second': 2.296, 'total_flos': 255125449820160.0, 'train_loss': 0.9282669204252737, 'epoch': 18.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: Unnamed: 0, text. If Unnamed: 0, text are not expected by `RobertaModelWithHeads.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 96
  Batch size = 16


{'eval_loss': 0.3070862591266632,
 'eval_runtime': 1.2838,
 'eval_samples_per_second': 74.779,
 'eval_steps_per_second': 4.674,
 'epoch': 18.0}

In [ ]:
model.set_active_adapters("docs")

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

The model 'RobertaModelWithHeads' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'LayoutLMv3ForSequenceClassification', 'LEDForSequenceClas

In [ ]:
classifier.save_pretrained('/content/drive/MyDrive/xmas_hack/models/adapterV4')

Configuration saved in /content/drive/MyDrive/xmas_hack/models/adapterV3/config.json
Model weights saved in /content/drive/MyDrive/xmas_hack/models/adapterV3/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/xmas_hack/models/adapterV3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/xmas_hack/models/adapterV3/special_tokens_map.json


In [ ]:
def choose_lable(label_preds):
  dict = {'Deliveries': 0,
      'Service_provision': 0,
      'Agreements': 0,
      'Rental': 0,
      'Purchase_and_sale': 0}
  for i in range(len(label_preds)):
    dict[label_preds[i][0]['label']] += 1
  return max(dict, key=dict.get)

In [ ]:
test_dataset = test_dataset.reset_index()
del test_dataset['index']

In [ ]:
del test_dataset['level_0']

In [ ]:
test_dataset

,text_norm,label
0,"﻿ договор г . _ « _ » _200_г . гражданин _ , п...",3
1,"﻿ договор г . _ « _ » _200_ года . _ , именуем...",3
2,﻿ ( типовая форма ) договор купли - продажи № ...,4
3,﻿ договор 21 . 11 . 2011 № 5 г . москва общест...,2
4,"﻿ договор поставки № _ рф , г . челябинск "" _ ...",0
5,﻿ договор закупки № _ / г . « _ » _ 2019 г . о...,0
6,﻿ договор № цкп . _ г . москва « _ » _ 2018 г ...,1
7,"﻿ договор № _ дата , место заключения ( населе...",0
8,﻿ 1 проект договора договор № _ г . ставрополь...,0
9,﻿ форма 2 . 2 приложение № 4 к приказу № _от _...,3


In [ ]:
names = {'Deliveries': '0',
      'Service_provision': '1',
      'Agreements': '2',
      'Rental': '3',
      'Purchase_and_sale': '4'}
preds = []
for i in range(len(test_dataset)):
  labels_preds = []
  text = test_dataset['text_norm'][i]
  print(text)
  for k in range(0, len(text)//1000):
      labels_preds.append(classifier(text[k:k+1000]))
      k += 1000
  #print(labels_preds)
  preds.append(names[choose_lable(labels_preds)])
preds

﻿ договор г . _ « _ » _200_г . гражданин _ , проживающий по адресу : _ , документ , удостоверяющий личность_ , серия _ , № _ , выдан _ , именуемый в дальнейшем « арендодатель », с одной стороны и _ », именуемое в дальнейшем « арендатор » в лице генерального директора _ -. с другой стороны , заключили настоящий договор о нижеследующем : 1 . предмет договора 1 . 1 . арендодатель обязуется предоставить во временное пользование арендатору техническое оборудование , сопровождаемые всеми принадлежностями и технической документацией , именуемые в дальнейшем « оборудование », а арендатор принять и своевременно возвратить оборудование в исправном состоянии с учетом нормального износа в соответствии с номенклатурой , прилагаемой к настоящему договору и являющейся его неотъемлемой частью . 1 . 2 . доходы , полученные арендатором в результате использования арендованного оборудования , являются собственностью арендатора . 1 . 3 . оборудование передается в состоянии , пригодном для использования и и

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


﻿ договор г . _ « _ » _200_ года . _ , именуемое в дальнейшем арендодатель , в лице генерального директора _ , действующего на основании устава , с одной стороны , и _ , именуемое в дальнейшем арендатор , в лице _ , действующего на основании устава , с другой стороны , подписали настоящий договор о нижеследующем : 1 . предмет договора 1 . 1 арендодатель передает , а арендатор принимает в аренду имущественный комплекс , в состав которого включены следующие компоненты : - третий пролет трехпролетного цеха опытного завода № _ ; - здание проходной ; - здание компрессорной станции ; - благоустройство территории расположения указанных объектов ( забор , асфальтированный участок дороги , бетонная площадка ; - право пользование земельным участком ; - оборудование , наименование и количество которого , согласованы в приложении № 1 к настоящему договору . далее по тексту имущественный комплекс , обозначенный в настоящем пункте , именуется « имущественный комплекс ». 1 . 2 . расположение имуществ

['3',
 '3',
 '4',
 '2',
 '0',
 '0',
 '1',
 '0',
 '0',
 '3',
 '1',
 '1',
 '4',
 '2',
 '2',
 '4',
 '3',
 '1',
 '1',
 '1',
 '0',
 '0',
 '4',
 '2']

In [ ]:
from sklearn.metrics import f1_score
f1_score(list(test_dataset['label']), preds, average='weighted')

1.0

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('cointegrated/rubert-tiny2')

In [ ]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=18,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    output_dir="./training_output",
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    
)

In [ ]:
text = (''.join(open('/content/drive/MyDrive/xmas_hack/txt_data/b303afea664ed05cdbc73a8c3b31b09d.txt', 'r', encoding='utf-8').readlines()))

In [ ]:
def choose_lable(label_preds):
    dict = {'Deliveries': 0,
        'Service_provision': 0,
        'Agreements': 0,
        'Rental': 0,
        'Purchase_and_sale': 0}
    for i in range(len(label_preds)):
        dict[label_preds[i][0]['label']] += 1
    return max(dict, key=dict.get)

In [ ]:
def predict(text):
  names_russian = {'0': 'Договоры поставки',
                  '1': 'Договоры оказания услуг',
                  '2': 'Договоры подряда',
                  '3': 'Договоры аренды',
                  '4': 'Договоры купли-продажи'}

  names_english = {'Deliveries': '0',
                'Service_provision': '1',
                'Agreements': '2',
                'Rental': '3',
                'Purchase_and_sale': '4'}
  text = preprocess.preprocess_pipeline(text)
  preds = []
  for k in range(len(text)//1000):
    preds.append(classifier(text[k:k+1000]))
    k += 1000
    #print(classifier(text[k:k+1000]))
  
  return names_russian[names_english[choose_lable(preds)]]
  

In [ ]:
def predict_words(text):
  names_russian = {'0': 'Договоры поставки',
                  '1': 'Договоры оказания услуг',
                  '2': 'Договоры подряда',
                  '3': 'Договоры аренды',
                  '4': 'Договоры купли-продажи'}

  names_english = {'Deliveries': '0',
                'Service_provision': '1',
                'Agreements': '2',
                'Rental': '3',
                'Purchase_and_sale': '4'}
  text = preprocess.preprocess_pipeline(text)
  preds = []
  
  preds.append(classifier(text))
  #print(classifier(text))
  
  return names_russian[names_english[choose_lable(preds)]]

In [ ]:
predict(text)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Договоры оказания услуг'

In [ ]:
classifier('основании')

[{'label': 'Rental', 'score': 0.2701852023601532}]

In [ ]:
preprocess.preprocess_pipeline(text)

'\ufeff evaluation only . created with aspose . words . copyright 2003 - 2022 aspose pty ltd . договор г . _ « _ » _ 2022 г . _ в лице _ , действующего на основании _ , именуемый в дальнейшем « заказчик », с одной стороны , и гр . _ , паспорт : серия _ , № _ , выданный _ , проживающий по адресу : _ , именуемый в дальнейшем « исполнитель », с другой стороны , именуемые в дальнейшем « стороны », заключили настоящий договор , в дальнейшем « договор », о нижеследующем : 1 . предмет договора 1 . 1 . заказчик поручает , а исполнитель принимает на себя обязательство выполнить следующие работы ( оказать услуги ), указанные в п . 2 . 1 . 1 договора , а заказчик в свою очередь обязуется оплатить их стоимость в порядке , установленном настоящим договором . 1 . 2 . заказчик обязуется оплатить работы ( услуги ) исполнителя в объеме и на протяжении срока , которые установлены в разделе 3 данного договора . 1 . 3 . стороны обязуются обеспечивать конфиденциальность информации , которая стала им извест

### Important words

In [ ]:
important_words = {}
label = predict(text)
print(label)
list_of_words = text.split()
for i in range(0, len(list_of_words)):
  list_of_words[i] = preprocess.preprocess_pipeline(list_of_words[i])
  #print(predict_words(list_of_words[i]), list_of_words[i])
  if predict_words(list_of_words[i]) == label:
    important_words[list_of_words[i]] = classifier(list_of_words[i])[0]['score']
    #print(1)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Договоры оказания услуг


In [ ]:
{k: v for k, v in sorted(important_words.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
important_words

### Important sentences

In [ ]:
sentences = preprocess.preprocess_pipeline(text).split('.')

In [ ]:
important_sentences = {}
label = predict(text)
print(label)
list_of_sentences = preprocess.preprocess_pipeline(text).split('.')
for i in range(0, len(list_of_sentences)):
  list_of_sentences[i] = preprocess.preprocess_pipeline(list_of_sentences[i])
  #print(predict_words(list_of_words[i]), list_of_words[i])
  if predict_words(list_of_sentences[i]) == label:
    important_sentences[list_of_sentences[i]] = classifier(list_of_sentences[i])[0]['score']
    print(1)

In [ ]:
important_sentences

In [ ]:
{k: v for k, v in sorted(important_sentences.items(), key=lambda item: item[1], reverse=True)}

{'подписи сторон заказчик _ исполнитель _ created with an evaluation copy of aspose': 0.7306692600250244,
 'исполнитель выполняет работы ( оказывает услуги ), указанные в п': 0.7126068472862244,
 'заказчик обязан : l оказывать содействие исполнителю в выполнении им работы ; l уплачивать за исполнителя в установленном порядке обязательные страховые взносы на государственное социальное страхование в фонд социальной защиты населения министерства труда и социальной защиты российской федерации ; l предоставлять при необходимости место для выполнения работ по договору , соответствующее правилам охраны труда и требованиям техники безопасности ; l осуществлять подготовку ( обучение ), инструктаж , повышение квалификации и проверку знаний исполнителя по вопросам безопасных условий выполнения работ либо требовать документы , подтверждающие прохождение им подготовки ( обучения ), инструктажа , медицинского осмотра , если это необходимо для выполнения указанных в пункте 2': 0.6908685564994812,
 'ю